In [1]:
import skylink
import numpy as np
from astropy.table import Table
import scipy.stats as stats

Let's make a mock-up data to experiment with:

In [2]:
def tnormal(mu=None, sigma=None, n=None, lower=-0.5, upper=0.5):
    return np.clip(np.random.normal(np.repeat(mu, n), sigma), lower, upper)

n = 1_000_000
np.random.seed(2)
ra = np.random.uniform(4, 6, n)
dec = np.random.uniform(-1, 1, n)

cat_a = Table({'ra': ra, 'dec': dec})
cat_b = Table({'ra': np.append(ra+tnormal(0, 0.0004, n), ra+tnormal(0, 0.0001, n)),
               'dec': np.append(dec+tnormal(0, 0.0002, n), dec+tnormal(0, 0.0002, n))})

The above cell generates *n=1M* galaxies for `cat_a` and *2M* galaxies for `cat_b`. Now that you have the two catalogs as dictionaries, we can pass them to `skylink` and do an internal (i.e. `friends of friends`) match for the two catalogs combined.

In [3]:
results = skylink.match({'a': cat_a, 'b':cat_b},  # catalogs to match in a dictionary
                        linking_lengths = 0.75,   # the linking length in arcsecond (can specify multiple values but not tested yet)
                        graph_lib = 'networkit',  # the graph library to use (other options: 'igraph', 'networkx')
                        nprocs = 8,               # number of processors for parallelization (multiprocessing by default)
                        overlap = 1.0,            # in the units of `linking_length`
                        use_linked_mask = True,   # create a mask for objects in overlapping regions to save time stitching fragmented group ids
                        sort = False,             # sort/group the results at the end in terms of `group_id` and `row_index`
                        verbose = True,           # display extended information wile processing (does not include progressbars and busy indicators)
                        leave = False,            # leave the progressbars on the screen after they are done
                        return_pandas = False     # return a panda's datafreame instead of an astropy Table
                                                  # and some more options ...
)

✔ Running 8 parallel jobs
✔ Mosaicking data    
✔ Growing the first KD-Tree    
✔ Used the same KD-Tree for the second set of coordinates since this is an internal match.
✔ Stored the KD-Tree in the cache.
✔ Finding all pairs of points whose distance is at most 0.75 arcsec    


✔ Created matching lists from KD-Trees
✔ Building the representative graph/network    


✔ Assigned group ids for each chunk by using connected components of the graphs
✔ Concatenating the results from different processes    
✔ Stitch fragmented group ids from different mosaic sets    
✔ Rearrange indices to be the same as original input    
✔ Success! Took 0:00:09 to execute.


After the run is finished, we can see the results.

In [4]:
results

row_index,catalog_key,group_id
int64,str1,int64
0,a,0
1,a,1
2,a,2
3,a,3
4,a,4
5,a,5
6,a,6
7,a,7
8,a,8


In case you forgot to set `sort=True`, you can always do so afterwards:

In [5]:
results.group_by('group_id') # or group_by(['group_id','row_index'] if needed

row_index,catalog_key,group_id
int64,str1,int64
0,a,0
1000000,b,0
1,a,1
1000001,b,1
2,a,2
856688,a,2
1000002,b,2
1856688,b,2
3,a,3


Also, if forgot to set `return_pandas=True`, you can convert your table to a pandas dataframe by:

In [6]:
results_df = results.to_pandas()
results_df

,row_index,catalog_key,group_id
0,0,a,0
1,1,a,1
2,2,a,2
3,3,a,3
4,4,a,4
...,...,...,...
2999995,1999995,b,1980068
2999996,1999996,b,1980069
2999997,1999997,b,1980070
2999998,1999998,b,970375


In order to check if the outputs of the two matchings are equal (including outputs from the benchmark `FoFCatalogMatching` package) you can use:

In [7]:
skylink.testing.assert_equal(results, results) # they, of course, give you equality

✔ The Tables are equal!


In [8]:
skylink.testing.assert_equal(results_df, results_df)

✔ The dataframes are equal!


Passing a path string (path to the pandas dataframe) is also allowed for one or both objects.

*Note: you can pickle pandas dataframes simply by doing `df.to_pickle("df.pkl")`*

In [9]:
skylink.testing.assert_equal(results_df, 'data/df.pkl') # they shouldn't be equal since I used a different linking length for the pickled one!

- Loading pandas pickle for the second object ...
✘ The dataframes are not equal!
